## This notebook inputs the missing values using KNN

In [6]:
import pandas as pd
from datetime import datetime
import numpy as np
from sklearn.impute import KNNImputer
from sklearn import preprocessing

#### Choose wether the imputation should be done on the data set with daily or weekly granularity

In [7]:
INPUT, OUTPUT = 'output/\CompleteWeeklyIndexes', 'output/\KNNWeeklyImputed'

In [8]:
df = pd.read_csv(INPUT+'.csv', parse_dates=True, index_col=0)
df_test = df.loc['2015-01-02':] #Selecting test data
df = df.loc['2000-01-01':'2015-01-01'] #Selecting training data

In [9]:
df_test.tail()

,sp500_Close,sp500_Volume,vix_Close,vix_Volume,dji_Close,dji_Volume,ndx_Close,ndx_Volume,n225_Close,n225_Volume,...,hsi_Close,hsi_Volume,n100_Close,n100_Volume,Overall EMV Tracker,infectious_daily_infect_emv_index,GPR,trade_US Trade Policy Uncertainty,trade_Japanese Trade Policy Uncertainty,trade_Trade Policy EMV Fraction
2020-02-14,3380.159912,3.398040e+09,13.680000,0.0,29398.080078,231000000.0,9623.580078,2.222240e+09,23687.589844,69100.0,...,27815.599609,1.445336e+09,1174.349976,204479200.0,NaN,1.23,NaN,NaN,NaN,NaN
2020-02-21,3337.750000,3.899270e+09,17.080000,0.0,28992.410156,311210000.0,9446.690430,2.743010e+09,23386.740234,62100.0,...,27308.810547,1.773613e+09,1164.170044,238290800.0,NaN,6.68,NaN,NaN,NaN,NaN
2020-02-28,2954.219971,8.563850e+09,40.110001,0.0,25409.359375,915990000.0,8461.830078,5.301170e+09,21142.960938,148500.0,...,26129.929688,3.191426e+09,1021.979980,655553200.0,24.443546,40.41,NaN,NaN,NaN,NaN
2020-03-06,2972.370117,6.552140e+09,41.939999,0.0,25864.779297,599780000.0,8530.339844,4.279850e+09,20749.750000,111700.0,...,26146.669922,2.646519e+09,994.770020,465581400.0,NaN,25.19,NaN,NaN,NaN,NaN
2020-03-13,2711.020020,8.258670e+09,57.830002,0.0,23185.619141,843080000.0,7995.259766,4.685890e+09,17431.050781,233400.0,...,24032.910156,5.495156e+09,800.750000,797087400.0,NaN,50.91,NaN,NaN,NaN,NaN


In [10]:
for i in range(0,30):
    #Scaled data has zero mean and unit variance
    scaler = preprocessing.StandardScaler()
    scaler.fit(df)
    array = scaler.transform(df)
    array_test = scaler.transform(df_test)
    df_scaled = pd.DataFrame(data=array, index=df.index, columns=df.columns)
    df_test_scaled = pd.DataFrame(data=array_test, index=df_test.index, columns=df_test.columns)
    
    imp = KNNImputer(n_neighbors=5, weights="uniform")
    imp.fit(df_scaled)
    df_imputed = imp.transform(df_scaled)
    df_test_imputed = imp.transform(df_test_scaled)
    df_final = pd.DataFrame(data=df_imputed, index=df.index, columns=df.columns)
    df_test_final = pd.DataFrame(data=df_test_imputed, index=df_test.index, columns=df_test.columns)
    # Descaling the data
    array_2 = scaler.inverse_transform(df_final)
    df_final2 = pd.DataFrame(data=array_2, index=df.index, columns=df.columns)

    array_test_2 = scaler.inverse_transform(df_test_final)
    df_test_final2 = pd.DataFrame(data=array_test_2, index=df_test.index, columns=df_test.columns)
    df_final2.to_csv(OUTPUT+"_"+str(i)+'_training.csv', index = True)
    df_test_final2.to_csv(OUTPUT+"_"+str(i)+'_test.csv', index = True)